In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\munna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
{
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "How are you?", "Is anyone there?", "Hey", "Hola", "Hi there"],
            "responses": ["Hello!", "Hi there!", "Hi! How can I help you?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["Goodbye!", "See you later!", "Bye! Have a nice day!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful"],
            "responses": ["You're welcome!", "Happy to help!", "Anytime!"]
        },
        {
            "tag": "software_issue",
            "patterns": ["My computer is slow", "I can't connect to the internet", "My software is not working", "I am getting an error message", "My application keeps crashing", "How to fix a blue screen error"],
            "responses": ["Try restarting your computer.", "Check your internet connection.", "Ensure your software is up to date.", "Reinstall the application.", "Please provide the error message for further assistance."]
        },
        {
            "tag": "software_installation",
            "patterns": ["How to install the software?", "I need help installing an application", "How to set up the program?", "Installation guide", "Install software"],
            "responses": ["Download the installer from the official website.", "Run the installer and follow the instructions.", "Ensure you have the necessary permissions.", "Refer to the installation guide provided with the software."]
        },
        {
            "tag": "password_reset",
            "patterns": ["How do I reset my password?", "I forgot my password", "Can't log in", "Password recovery", "Reset my password"],
            "responses": ["Click on the 'Forgot Password' link on the login page.", "Enter your email address to receive a reset link.", "Follow the instructions in the email to reset your password."]
        },
        {
            "tag": "software_update",
            "patterns": ["How to update the software?", "Update my application", "New version available", "Software upgrade", "Install the latest version"],
            "responses": ["Go to the settings or help menu and select 'Check for updates'.", "Visit the official website to download the latest version.", "Ensure your system meets the requirements for the update.", "Follow the on-screen instructions to complete the update."]
        },
        {
            "tag": "health_advice",
            "patterns": ["I have a headache", "What should I do if I have a cold?", "How to treat a fever?", "Health tips", "Medical advice", "I feel unwell"],
            "responses": ["Drink plenty of water and rest.", "Take over-the-counter medication if necessary.", "If symptoms persist, consult a healthcare professional.", "Maintain a healthy diet and exercise regularly."]
        },
        {
            "tag": "appointment",
            "patterns": ["How to book an appointment?", "I need to see a doctor", "Schedule a consultation", "Doctor's appointment", "Book a visit"],
            "responses": ["Visit our website to book an appointment online.", "Call our clinic to schedule a consultation.", "Provide your details and preferred time for the appointment.", "You will receive a confirmation via email or phone."]
        },
        {
            "tag": "emergency",
            "patterns": ["I have a medical emergency", "Call an ambulance", "Urgent help needed", "What to do in an emergency?", "Need immediate assistance"],
            "responses": ["Call emergency services immediately.", "Stay calm and follow the operator's instructions.", "Ensure you are in a safe location.", "Provide as much information as possible about the situation."]
        },
        {
            "tag": "healthy_lifestyle",
            "patterns": ["Tips for a healthy lifestyle", "How to stay fit?", "Diet and exercise advice", "Healthy living", "Wellness tips"],
            "responses": ["Maintain a balanced diet rich in fruits and vegetables.", "Exercise regularly, at least 30 minutes a day.", "Get adequate sleep, around 7-8 hours per night.", "Stay hydrated by drinking plenty of water."]
        }
    ]
}


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hello',
    'How are you?',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hi there'],
   'responses': ['Hello!', 'Hi there!', 'Hi! How can I help you?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['Goodbye!', 'See you later!', 'Bye! Have a nice day!']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ["You're welcome!", 'Happy to help!', 'Anytime!']},
  {'tag': 'software_issue',
   'patterns': ['My computer is slow',
    "I can't connect to the internet",
    'My software is not working',
    'I am getting an error message',
    'My application keeps crashing',
    'How to fix a blue screen error'],
   'responses': ['Try restarting your computer.',
    'Check your internet connection.',
    'Ensure your software is up to date.',
    'Reinstall the application.',
    'Please provide the error message for further assistance.']},
 

In [3]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('D:\Datascience\chatbot\intents.json').read())

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(f"Words: {words}")
print(f"Classes: {classes}")

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])


Words: ["'s", 'a', 'advice', 'am', 'ambulance', 'an', 'and', 'anyone', 'application', 'appointment', 'are', 'assistance', 'available', 'blue', 'book', 'bye', 'ca', 'call', 'cold', 'computer', 'connect', 'consultation', 'crashing', 'diet', 'do', 'doctor', 'emergency', 'error', 'exercise', 'feel', 'fever', 'fit', 'fix', 'for', 'forgot', 'getting', 'goodbye', 'guide', 'have', 'headache', 'health', 'healthy', 'hello', 'help', 'helpful', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'immediate', 'in', 'install', 'installation', 'installing', 'internet', 'is', 'keep', 'later', 'latest', 'lifestyle', 'living', 'log', 'medical', 'message', 'my', "n't", 'need', 'needed', 'new', 'not', 'password', 'program', 'recovery', 'reset', 'schedule', 'screen', 'see', 'set', 'should', 'slow', 'software', 'stay', 'thank', 'thanks', 'that', 'the', 'there', 'tip', 'to', 'treat', 'unwell', 'up', 'update', 'upgrade', 'urgent', 'version', 'visit', 'wellness', 'what', 'working', 'you']
Classes: ['appointment', 'emergenc

In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200


C:\Users\munna\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\munna\AppData\Roaming\Python\Python311\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0495 - loss: 2.3963    
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0743 - loss: 2.3946     
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4142 - loss: 2.2674 
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0539 - loss: 2.3342     
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3184 - loss: 2.2071 
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2837 - loss: 2.1346 
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1953 - loss: 2.2323     
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4130 - loss: 1.9829 
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3909 - loss: 1.9642 
Epoch 10/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - accuracy: 0.4686 - loss: 1.9234
Epoch 11/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4672 - loss: 1.7723 
Epoch 12/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

In [5]:
import pickle

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, intents)
    return res


In [6]:
import streamlit as st
import json
import random
import numpy as np
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

nltk.download('punkt')
nltk.download('wordnet')

# Load necessary files
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, intents)
    return res

# Streamlit UI
st.title("Chatbot")
st.write("Welcome to the chatbot! Type a message and get a response.")

user_input = st.text_input("You:", "")
if st.button("Send"):
    if user_input:
        response = chatbot_response(user_input)
        st.write(f"Chatbot: {response}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\munna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-07-22 09:55:15.887 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
